In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro
import os
import statistics
import seaborn as sns
from scipy.stats import norm

import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual
from Project.Utils.visualize import pvalue, search_indicators


In [2]:
#One dataframe per country

df= pd.read_csv (os.getcwd()+'/Output/GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = ['Country']
aux_df = pd.DataFrame()

#List all the countries, none repeated
countries = set(df['Country'].to_list())

country_dict = {}
corr_dict = {}

for country in countries:

    #Get the DataFrame for a given country
    country_df = df.loc[df['Country'] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {'GDP': country}).drop(index = ['Year', 'GDP'])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format and show it
corr_df = corr_df.transpose()
corr_df


,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Japan,-0.491802,NaN,NaN,0.910719,0.393421,-0.375865,0.441033,-0.443549,0.349423,-0.375301,...,-0.554870,-0.566014,-0.034650,-0.045268,0.367248,-0.466609,-0.178459,-0.574905,0.016904,0.163330
Guinea,0.498159,NaN,NaN,0.174019,-0.694951,-0.939092,-0.914487,NaN,0.935467,-0.879412,...,0.907336,-0.954342,0.293413,0.625618,0.694268,0.873440,0.419932,NaN,NaN,0.968902
Jordan,0.929244,-0.828580,NaN,0.979891,-0.925262,-0.969768,-0.750168,NaN,0.987968,-0.989961,...,0.044476,-0.992862,0.573174,0.940235,0.116714,-0.936668,NaN,NaN,NaN,0.994118
Solomon Islands,-0.399197,NaN,NaN,NaN,NaN,-0.919521,-0.981036,NaN,0.984354,-0.947507,...,0.225781,-0.987071,0.505555,0.949567,-0.725721,0.667742,NaN,NaN,NaN,0.981798
India,-0.700650,0.838653,NaN,0.979770,0.338557,-0.984446,-0.922763,-0.945650,0.987074,-0.952978,...,-0.987524,-0.989709,0.514362,-0.814355,0.701925,0.927380,NaN,NaN,NaN,0.987350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Colombia,-0.928724,NaN,0.758967,0.641759,0.634779,-0.901693,0.659420,-0.837504,0.859863,-0.874104,...,-0.678381,-0.856736,0.782616,0.653603,0.535807,0.568786,0.628709,0.632915,0.773963,0.823003
China,NaN,NaN,NaN,0.949529,-0.860912,-0.362204,0.776483,-0.927046,0.982323,-0.868190,...,-0.550435,-0.983006,-0.713696,-0.956523,0.825297,0.767233,NaN,NaN,NaN,0.982877
Greece,-0.854153,0.114222,0.408677,0.862916,-0.425234,0.588829,-0.253547,0.524593,0.171029,0.428454,...,0.048550,-0.072185,0.292634,0.380068,0.513542,0.163758,NaN,0.236961,0.633141,0.155726
Thailand,0.036810,-0.919643,NaN,0.881927,0.172728,-0.979122,0.921562,-0.944235,0.987648,-0.971913,...,-0.955077,-0.985585,-0.799532,-0.752925,0.571812,0.848803,NaN,-0.359601,-0.495271,0.979655


In [3]:
import ipywidgets as widgets
import plotly.express as px


indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)


def globalGrapgh(indicator):
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='Reds',
                    width=700, height=500)
    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    
    
widgets.interactive(globalGrapgh, indicator = indicator) 

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

In [6]:
def tableOut(Threshold, Country):
    
    df = search_indicators(Threshold, corr_df)[Country]
    df  = pd.merge(df, pvalue(Country), left_index=True, right_index=True, how = 'inner')
    
    
    left = pd.Series([0.05], index=['P-value'])
    left2 = pd.Series([-1], index=['GDP Correlation'])
    left3 = pd.Series([0], index=['GDP Correlation'])
    
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Correlation'])\
        .format('{:,.12f}', subset = ['P-value'], )
        #.format('{:12.3e}', subset = ['P-value'], )


    display(df)
    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…